# Imports

In [ ]:
from scheduling_funtions import *
from constraints import *
from data import *
from model import *


# Creating Data

In [ ]:
staff = create_data(staff_list)
ft_staff = filter_data(staff, ft_only_staff_mask)
midnight_staff = filter_data(staff, midnight_staff_mask)
six_month_new_staff = filter_data(staff, staff_in_first_6_months_mask)

shifts = create_data(shift_list)
midnight_shifts = filter_data(shifts, midnight_shifts_mask)
not_midnight_shifts = list_difference(shifts, midnight_shifts)
late_shifts = filter_data(shifts, late_shifts_mask)
day_shifts = filter_data(shifts, day_shifts_mask)
afternoon_shifts = filter_data(shifts, afternoon_shifts_mask)
ft_shifts = filter_data(shifts, ft_shifts_mask)
not_ft_shifts = list_difference(shifts, ft_shifts)
on_call_shifts = filter_data(shifts, on_call_shifts_mask)
not_on_call_shifts = list_difference(shifts, on_call_shifts)
after_5_shifts = late_shifts + midnight_shifts
after_930_shifts = list_difference(shifts, day_shifts)

days, first_day = create_date_range(1, 1, 2021)
fridays = fridays(days, first_day)
weekdays = mondays(days, first_day) + tuesdays(days, first_day) + wednesdays(days, first_day) + thursdays(days, first_day) + fridays
saturdays = saturdays(days, first_day)
sundays = sundays(days, first_day)
weekends = saturdays + sundays

# Creating the Model

In [ ]:
model = create_model()
staff_works_shift_on_day = create_model_variables(model, "staff_works_shift_on_day", staff, days, shifts)
staff_works_day = create_model_variables_with_sum(model, "staff_works_day", staff_works_shift_on_day, staff, days, shifts)
staff_works_afternoon_shift = create_model_variables_with_sum(model, "staff_works_afternoon_shift", staff_works_shift_on_day, staff, days, afternoon_shifts)
staff_works_midnight_shift = create_model_variables_with_sum(model, "staff_works_midnight_shift", staff_works_shift_on_day, staff, days, midnight_shifts)
staff_works_on_call_shift = create_model_variables_with_sum(model, "staff_works_on_call_shift", staff_works_shift_on_day, staff, days, on_call_shifts)
staff_works_ft_shift = create_model_variables_with_sum(model, "staff_works_ft_shift", staff_works_shift_on_day, staff, days, ft_shifts)
staff_works_late_shift = create_model_variables_with_sum(model, "staff_works_late_shift", staff_works_shift_on_day, staff, days, late_shifts)
staff_works_after_5_shift = create_model_variables_with_sum(model, "staff_works_after_5_shift", staff_works_shift_on_day, staff, days, after_5_shifts)
staff_works_after_930_shift = create_model_variables_with_sum(model, "staff_works_after_930_shift", staff_works_shift_on_day, staff, days, after_930_shifts)
staff_works_day_shift = create_model_variables_with_sum(model, "staff_works_after_930_shift", staff_works_shift_on_day, staff, days, day_shifts)
# staff_productivities = create_staff_variables(model, "staff_productivity", staff_productivity_mask, staff_works_shift_on_day, staff, days, shifts)

obj_int_vars, obj_int_coeffs = empty_minimize_constraints()
obj_bool_vars, obj_bool_coeffs = empty_minimize_constraints()

# Adding Constraints to the Model

In [ ]:
max_days_worked_hard_max = 7
max_days_worked_soft_max = 3
max_days_worked_max_cost = MAX

min_days_off_after_midnight_hard_min = 2
min_days_off_after_midnight_soft_min = 3
min_days_off_after_midnight_min_cost = HIGH

max_midnights_in_a_row_hard_max = 2
max_midnights_in_a_row_soft_min = 1
max_midnights_in_a_row_max_cost = MAX

no_late_shift_before_time_off_hard_min = 1
no_late_shift_before_time_off_soft_min = 0
no_late_shift_before_time_off_min_cost = NONE

on_call_rules_before_hard_min = 1
on_call_rules_before_soft_min = 0
on_call_rules_before_min_cost = NONE

on_call_rules_after_hard_min = 1
on_call_rules_after_soft_min = 0
on_call_rules_after_min_cost = NONE

days_off_after_consecutive_shifts_hard_min = 2
days_off_after_consecutive_shifts_soft_min = 0
days_off_after_consecutive_shifts_min_cost = NONE

days_off_between_late_and_day_shifts_hard_min = 0
days_off_between_late_and_day_shifts_soft_min = 3
days_off_between_late_and_day_shifts_min_cost = HIGH

days_off_between_late_and_afternoon_shifts_hard_min = 0
days_off_between_late_and_afternoon_shifts_soft_min = 2
days_off_between_late_and_afternoon_shifts_min_cost = HIGH

late_shifts_in_a_row_hard_max = 7
late_shifts_in_a_row_soft_max = 3
late_shifts_in_a_row_max_cost = MAX

late_shifts_in_weeks_hard_min = 0
late_shifts_in_weeks_soft_min = 0
late_shifts_in_weeks_min_cost = NONE
late_shifts_in_weeks_hard_max = 14
late_shifts_in_weeks_soft_max = 5
late_shifts_in_weeks_max_cost = MAX

avoid_consecutive_ft_shifts_hard_max = 2
avoid_consecutive_ft_shifts_soft_max = 1
avoid_consecutive_ft_shifts_max_cost = MID

no_nightshifts_before_weekend_off_hard_min = 0
no_nightshifts_before_weekend_off_soft_min = 1
no_nightshifts_before_weekend_off_min_cost = HIGH

minimize_split_weekends_cost = MAX

equalize_weekends_cost = 4
equalize_night_shifts_cost = 4
equalize_late_shifts_cost = 4
equalize_day_shifts_cost = 2
equalize_afternoon_shifts_cost = 2
equalize_weekdays_cost = 1


In [ ]:
all_shifts_taken(model,
                staff_works_shift_on_day,
                staff,
                days,
                shifts)

max_days_worked(model,
                staff_works_day,
                staff,
                days,
                max_days_worked_hard_max,
                max_days_worked_soft_max,
                max_days_worked_max_cost,
                obj_bool_vars,
                obj_bool_coeffs)

min_days_off_after_midnight(model,
                            staff_works_day,
                            staff_works_midnight_shift, 
                            staff, 
                            days, 
                            min_days_off_after_midnight_hard_min, 
                            min_days_off_after_midnight_soft_min, 
                            min_days_off_after_midnight_min_cost, 
                            obj_bool_vars, 
                            obj_bool_coeffs)

midnight_physicians(model,
                    staff_works_shift_on_day,
                    midnight_staff,
                    days,
                    not_midnight_shifts)

no_midnights_within_six_months(model,
                                staff_works_shift_on_day,
                                six_month_new_staff,
                                days,
                                midnight_shifts)

max_midnights_in_a_row(model,
                        staff_works_midnight_shift,
                        staff,
                        days,
                        max_midnights_in_a_row_hard_max,
                        max_midnights_in_a_row_soft_min,
                        max_midnights_in_a_row_max_cost,
                        obj_bool_vars,
                        obj_bool_coeffs)

ft_physicians(model,
            staff_works_shift_on_day,
            ft_staff,
            days,
            not_ft_shifts)

no_late_shift_before_time_off(model,
                            staff_works_day,
                            staff_works_after_5_shift,
                            staff,
                            no_late_shift_before_time_off_hard_min,
                            no_late_shift_before_time_off_soft_min,
                            no_late_shift_before_time_off_min_cost,
                            obj_bool_vars,
                            obj_bool_coeffs)
                            
on_call_rules_before(model,
                    staff_works_on_call_shift,
                    staff_works_after_5_shift,
                    staff,
                    days,
                    on_call_rules_before_hard_min,
                    on_call_rules_before_soft_min,
                    on_call_rules_before_min_cost,
                    obj_bool_vars,
                    obj_bool_coeffs)

on_call_rules_after(model,
                    staff_works_on_call_shift,
                    staff_works_day_shift,
                    staff,
                    days,
                    on_call_rules_after_hard_min,
                    on_call_rules_after_soft_min,
                    on_call_rules_after_min_cost,
                    obj_bool_vars,
                    obj_bool_coeffs)

days_off_after_consecutive_shifts(model,
                                staff_works_day,
                                staff,
                                days,
                                days_off_after_consecutive_shifts_hard_min,
                                days_off_after_consecutive_shifts_soft_min,
                                days_off_after_consecutive_shifts_min_cost,
                                obj_bool_vars,
                                obj_bool_coeffs)


In [ ]:
transitions_constraints(model,
                        staff_works_shift_on_day,
                        staff,
                        days,
                        obj_bool_vars,
                        obj_bool_coeffs)

# days_off_between_late_and_day_shifts(model,
#                                     staff_works_day,
#                                     staff_works_day_shift,
#                                     staff_works_late_shift,
#                                     staff,
#                                     days,
#                                     days_off_between_late_and_day_shifts_hard_min,
#                                     days_off_between_late_and_day_shifts_soft_min,
#                                     days_off_between_late_and_day_shifts_min_cost,
#                                     obj_bool_vars,
#                                     obj_bool_coeffs)

# days_off_between_late_and_afternoon_shifts(model,
#                                             staff_works_day,
#                                             staff_works_afternoon_shift,
#                                             staff_works_late_shift,
#                                             staff,
#                                             days,
#                                             days_off_between_late_and_afternoon_shifts_hard_min,
#                                             days_off_between_late_and_afternoon_shifts_soft_min,
#                                             days_off_between_late_and_afternoon_shifts_min_cost,
#                                             obj_bool_vars,
#                                             obj_bool_coeffs)

late_shifts_in_a_row(model,
                    staff_works_late_shift,
                    staff,
                    days,
                    late_shifts_in_a_row_hard_max,
                    late_shifts_in_a_row_soft_max,
                    late_shifts_in_a_row_max_cost,
                    obj_bool_vars,
                    obj_bool_coeffs)

late_shifts_in_weeks(model,
                        staff_works_late_shift,
                        staff,
                        days,
                        late_shifts_in_weeks_hard_min,
                        late_shifts_in_weeks_soft_min,
                        late_shifts_in_weeks_min_cost,
                        late_shifts_in_weeks_hard_max,
                        late_shifts_in_weeks_soft_max,
                        late_shifts_in_weeks_max_cost,
                        obj_int_vars,
                        obj_int_coeffs)

avoid_consecutive_ft_shifts(model,
                        staff_works_ft_shift,
                        staff,
                        days,
                        avoid_consecutive_ft_shifts_hard_max,
                        avoid_consecutive_ft_shifts_soft_max,
                        avoid_consecutive_ft_shifts_max_cost,
                        obj_int_vars,
                        obj_int_coeffs)

In [ ]:
equalize_weekends(model,
                    staff_works_day,
                    staff,
                    weekends,
                    equalize_weekends_cost,
                    triangle_costs(len(shifts), len(weekends), len(staff)),
                    obj_int_vars,
                    obj_int_coeffs)

minimize_split_weekends(model,
                        staff_works_day,
                        staff,
                        saturdays,
                        sundays,
                        minimize_split_weekends_cost,
                        obj_int_vars,
                        obj_int_coeffs)

equalize_night_shifts(model,
                        staff_works_midnight_shift,
                        staff,
                        days,
                        equalize_night_shifts_cost,
                        triangle_costs(len(midnight_shifts), len(days), len(staff)),
                        obj_int_vars,
                        obj_int_coeffs)

no_nightshifts_before_weekend_off(model,
                                staff_works_day,
                                staff_works_after_5_shift,
                                staff,
                                fridays,
                                saturdays,
                                no_nightshifts_before_weekend_off_hard_min,
                                no_nightshifts_before_weekend_off_soft_min,
                                no_nightshifts_before_weekend_off_min_cost,
                                obj_bool_vars,
                                obj_bool_coeffs)

equalize_late_shifts(model,
                    staff_works_afternoon_shift,
                    staff,
                    days,
                    equalize_late_shifts_cost,
                    triangle_costs(len(late_shifts), len(days), len(staff)),
                    obj_int_vars,
                    obj_int_coeffs)

equalize_day_shifts(model,
                    staff_works_day_shift,
                    staff,
                    days,
                    equalize_day_shifts_cost,
                    triangle_costs(len(day_shifts), len(days), len(staff)),
                    obj_int_vars,
                    obj_int_coeffs)

equalize_afternoon_shifts(model,
                            staff_works_afternoon_shift,
                            staff,
                            days,
                            equalize_afternoon_shifts_cost,
                            triangle_costs(len(afternoon_shifts), len(days), len(staff)),
                            obj_int_vars,
                            obj_int_coeffs)

equalize_weekdays(model,
                    staff_works_day,
                    staff,
                    weekdays,
                    equalize_weekdays_cost,
                    triangle_costs(len(shifts), len(weekdays), len(staff)),
                    obj_int_vars,
                    obj_int_coeffs)

In [ ]:
apply_requests(staff_works_shift_on_day,
                staff_works_day,
                obj_int_vars,
                obj_int_coeffs,
                requests)
# apply_productivity(staff, 3)

# Solving the Model

In [ ]:
# All constraints together takes 1.5 hours to find a feasible solution
import time

start = time.time()

model.Minimize(
        sum(obj_bool_vars[i] * obj_bool_coeffs[i]
            for i in range(len(obj_bool_vars))) +
            sum(obj_int_vars[i] * obj_int_coeffs[i]
            for i in range(len(obj_int_vars))))

solver = create_solver(240)
status = solver.Solve(model)

print(solver.StatusName(status))

if status == FEASIBLE or status == OPTIMAL:

    print('Maximum of objective function: %i' % solver.ObjectiveValue())
    print()
    for d in days:
        print('Day', d)
        for s in shifts:
            for m in staff:
                if solver.Value(staff_works_shift_on_day[(m, d, s)]) == 1:
                    print(staff_list[m], 'works shift', shift_list[s])
        print()

end = time.time()
print(f"time elapsed: {end - start}")

# Tests

In [ ]:
staff_works_day_results = {}
staff_works_midnight_shift_results = {}
staff_works_late_shift_results = {}
staff_works_day_shift_results = {}
staff_works_shift_on_day_results = {}
staff_works_ft_shift_results = {}
staff_works_after_5_shift_results = {}
staff_works_on_call_shift_results = {}
staff_works_after_930_shift_results = {}
for m in staff:
    staff_works_day_results[m] = [solver.Value(staff_works_day[m, d]) for d in days]
    staff_works_midnight_shift_results[m] = [solver.Value(staff_works_midnight_shift[m, d]) for d in days]
    staff_works_late_shift_results[m] = [solver.Value(staff_works_late_shift[m, d]) for d in days]
    staff_works_day_shift_results[m] = [solver.Value(staff_works_day_shift[m, d]) for d in days]
    staff_works_ft_shift_results[m] = [solver.Value(staff_works_ft_shift[m, d]) for d in days]
    staff_works_after_5_shift_results[m] = [solver.Value(staff_works_after_5_shift[m,d]) for d in days]
    staff_works_on_call_shift_results[m] = [solver.Value(staff_works_on_call_shift[m,d]) for d in days]
    staff_works_after_930_shift_results[m] = [solver.Value(staff_works_after_930_shift[m,d]) for d in days]
    for d in days:
        staff_works_shift_on_day_results[(m, d)] = [solver.Value(staff_works_shift_on_day[(m, d, s)]) for s in shifts]

## Hard Constraints

In [ ]:
# No two doctors in the same shift on the same day
# All shifts should be taken by doctors
for d in days:
    num_staff_per_shift = zip(*[staff_works_shift_on_day_results[m, d] for m in staff])
    assert(list(sum(column) for column in num_staff_per_shift) == [1] * len(shifts))
  
#No two shifts same day
for key, result in staff_works_shift_on_day_results.items():
    assert(sum(result) <= 1)

# Maximum work 7 consecutive days
for key, result in staff_works_day_results.items():
    assert(not detect_pattern(result, "11111111"))

# # 2 days off after last midnight (except on call shift).
for key in staff_works_midnight_shift_results.keys() and staff_works_day_results.keys():
    if not midnight_staff_mask[key]:
        for idx in days[:-3]:
            if staff_works_midnight_shift_results[key][idx] == 1 and staff_works_midnight_shift_results[key][idx+1] == 0:
                assert(sum(staff_works_day_shift_results[key][idx+1:idx+3]) == 0)

# Certain staff work midnights only
for key, result in staff_works_shift_on_day_results.items():
    if midnight_staff_mask[key[0]]:
        assert(result == [0,0,0,0,0,0,0,0,0,0,1] or result == [0] * len(shifts))

# No midnights for staff in their first 6 months (need way to indicate when physician is in first 6 months of practice)
for key, result in staff_works_midnight_shift_results.items():
    if (staff_in_first_6_months_mask[key]):
        assert(sum(result) == 0)

# Maximum 2 midnights in a row (except for several physicians who only work midnights)
for key, result in staff_works_midnight_shift_results.items():
    if not midnight_staff_mask[key]:
        assert(not detect_pattern(result, "111"))

# Certain physicians work only FT shift (0730,1530 shift)
for key, result in staff_works_day_results.items():
    print(key)
    print(result)
    if ft_only_staff_mask[key[0]]:
        assert(result == [0,1,0,0,0,0,0,0,0,0,0] or result == [0,0,0,0,0,1,0,0,0,0,0] or result == [0] * len(shifts))

# No 2000, 2200, or midnight shift prior to day requested off
for staff2, day2, shift2, coef in requests:
    if staff_works_day_results[staff2][day2] != 0 and shift2 == -1 and coef > 0:
        assert(staff_works_after_5_shift_results[staff2][day2-1] == 0)

# Physicians can work the 0930 shifts or earlier prior to working on call. They can work starting no earlier than 11 the day after on call.
# On call shift - day after rules
for key, result in staff_works_on_call_shift_results.items() and staff_works_day_results.items():
    for idx, elem in enumerate(on_call_results[key]):
        if idx > 0 and staff_works_on_call_shift_results[key][idx]:
            assert(staff_works_day_results[key][idx - 1] == 0 or staff_works_shift_on_day_results[key][idx-1] == 1)
        if idx < len(staff_works_on_call_shift_results[key]) - 1 and staff_works_on_call_shift_results[key][idx]:
            assert(staff_works_shift_on_day_results[key][idx + 1] == 0)

# 2 days off after 3 to 7 days of work in a row
for key, result in staff_works_day_results.items():
    assert(not detect_pattern(result, "11101"))

print("ALL HARD CONSTRAINTS SATISFIED")

## Pie in the sky

In [ ]:
#Ensure requests work as intended
number_of_requests_respected = 0
number_of_requests_disrespected = 0

for staff2, day2, shift2, coef in requests:
    if shift2 == -1 and coef > 0:
        if solver.Value(staff_works_day_results[staff2, day2]):
            number_of_requests_disrespected += 1
        else:
            number_of_requests_respected += 1
    elif shift2 == -1 and coef < 0:
        if solver.Value(staff_works_day_results[staff2, day2]):
            number_of_requests_respected += 1
        else:
            number_of_requests_disrespected += 1
    elif coef > 0:
        if solver.Value(staff_works_shift_on_day_results[staff2, day2, shift2]):
            number_of_requests_disrespected += 1
        else:
            number_of_requests_respected += 1
    elif coef < 0:
        if solver.Value(staff_works_shift_on_day_results[staff2, day2, shift2]):
            number_of_requests_respected += 1
        else:
            number_of_requests_disrespected += 1

print(f"number of requests respected: {number_of_requests_respected}")
print(f"number of requests disrespected: {number_of_requests_disrespected}")

In [ ]:
# #Ensure requests work as intended
# productivity_under = 0
# productivity_over = 0

# for d in days:
#     for wind in window(shifts[:-1], 3):
#         prod = [solver.Value(productivities[m,d,s])
#         for m in staff 
#         for s in wind]
#         sums = sum(prod)
#         if sums < 6:
#             productivity_under += 1
#         if sums > 15:
#             productivity_over += 1

# print(f"number of times theres too little productivity: {productivity_under}")
# print(f"number of times theres too much productivity: {productivity_over}")

## Soft Constraints

In [ ]:
# General principle avoid shift times changing too much day to day
# Shifts should have same start time to 2.5 hours later compared to previous shift (the 2 hours later can be relaxed to 3,4 perhaps)
# No shifts that start more than 1.5 hours earlier than the shift on the previous day
penalized_transitions = []

for shift in list(permutations(shift_list[:-1], 2)):
    t1 = float(shift[0][0:2] + '.' + shift[0][2:4])
    t2 = float(shift[1][0:2] + '.' + shift[1][2:4])
    if t2 - t1 > 2.5:
        penalized_transitions.append(((shift_list.index(shift[0]), shift_list.index(shift[1])), 1))
    elif t2 - t1 < -1.5:
        penalized_transitions.append(((shift_list.index(shift[0]), shift_list.index(shift[1])), 3))


transitions = {elem[0]: elem[1] for elem in penalized_transitions}
jumps_forward_25 = 0
jumps_backwards_15 = 0
previous_shift = -1
for key, result in staff_works_shift_on_day_results.items():
    if previous_shift == -1:
        try:
            previous_shift = result.index(1)
        except:
            previous_shift = -1
    else:
        try:
            ans = transitions[(previous_shift, result.index(1))]
            if ans == 1:
                jumps_forward_25 += 1
            elif ans == 3:
                jumps_backwards_15 += 1
            previous_shift = result.index(1)
        except:
            previous_shift = -1

    if key[1] == len(days) - 1:
        previous_shift = -1 

print(f"Number of times shifts jump forwards 2.5 hours: {jumps_forward_25}")
print(f"Number of times shifts jump backwards 1.5 hours: {jumps_forward_25}")
print()

In [ ]:
# 3 days off after midnight. 4 even better
two_days_after_midnight = 0
three_days_after_midnight = 0
four_days_after_midnight = 0
five_days_after_midnight = 0
for key in midnights_worked_results.keys() and days_worked_results.keys():
    for idx in days[:-4]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+3]) == 0 and sum(days_worked_results[key][idx+1:idx+4]) == 1:
                two_days_after_midnight += 1
    for idx in days[:-5]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+4]) == 0 and sum(days_worked_results[key][idx+1:idx+5]) == 1:
                three_days_after_midnight += 1
    for idx in days[:-6]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+5]) == 0 and sum(days_worked_results[key][idx+1:idx+6]) == 1:
                four_days_after_midnight += 1
    for idx in days[:-6]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+6]) == 0:
                five_days_after_midnight += 1
print(f"Number of times an employee gets 2 days off after midnights: {two_days_after_midnight}")
print(f"Number of times an employee gets 3 days off after midnights: {three_days_after_midnight}")
print(f"Number of times an employee gets 4 days off after midnights: {four_days_after_midnight}")
print(f"Number of times an employee gets 5 days off after midnights: {five_days_after_midnight}")
print()

In [ ]:
#Max 1 midnight in a row
two_midnights_in_a_row = 0
for key, result in midnights_worked_results.items():
    two_midnights_in_a_row += detect_pattern_soft(result, '11')
print(f"Number of times an employee works 2 midnights in a row: {two_days_after_midnight}")
print()

In [ ]:
# 3 days off when transitioning from late shift to day shift 
one_days_off_from_late_to_day = 0
two_days_off_from_late_to_day = 0
three_days_off_from_late_to_day = 0
for key in day_shifts_worked_results.keys() and lates_worked_results.keys():
    for idx, elem in enumerate(lates_worked_results[key][:-2]):
        if elem == 1 and day_shifts_worked_results[key][idx + 2]:
            if sum(days_worked_results[key][idx+1:idx+2]) == 0:
                one_days_off_from_late_to_day += 1
    for idx, elem in enumerate(lates_worked_results[key][:-3]):
        if elem == 1 and day_shifts_worked_results[key][idx + 3]:
            if sum(days_worked_results[key][idx+1:idx+3]) == 0:
                two_days_off_from_late_to_day += 1
    for idx, elem in enumerate(lates_worked_results[key][:-4]):
        if elem == 1 and day_shifts_worked_results[key][idx + 4]:
            if sum(days_worked_results[key][idx+1:idx+4]) == 0:
                three_days_off_from_late_to_day += 1
print(f"Number of times there is a day shift within 1 days of a late shift: {one_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 2 days of a late shift: {two_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 3 days of a late shift: {three_days_off_from_late_to_day}")
print()

In [ ]:
# 2 days off when transitioning from late shift to afternoon shift (although this transition should be avoided) 
one_days_off_from_late_to_afternoon = 0
two_days_off_from_late_to_afternoon = 0
for key in day_shifts_worked_results.keys() and lates_worked_results.keys():
    for idx, elem in enumerate(lates_worked_results[key][:-2]):
        if elem == 1 and day_shifts_worked_results[key][idx + 2]:
            if sum(days_worked_results[key][idx+1:idx+2]) == 0:
                one_days_off_from_late_to_afternoon += 1
    for idx, elem in enumerate(lates_worked_results[key][:-3]):
        if elem == 1 and day_shifts_worked_results[key][idx + 3]:
            if sum(days_worked_results[key][idx+1:idx+3]) == 0:
                two_days_off_from_late_to_afternoon += 1
print(f"Number of times there is a afternoon shift within 1 days of a late shift: {one_days_off_from_late_to_afternoon}")
print(f"Number of times there is a afternoon shift within 2 days of a late shift: {two_days_off_from_late_to_afternoon}")
print()

In [ ]:
# 3 late shifts in a row maximum - late shifts are 1800, 2000, 2200. The ability to set what a late shift is would be great
three_days_in_a_row_late = 0
four_days_in_a_row_late = 0
five_days_in_a_row_late = 0
for key, result in lates_worked_results.items():
    three_days_in_a_row_late += detect_pattern_soft(result, '111')
    four_days_in_a_row_late += detect_pattern_soft(result, '1111')
    five_days_in_a_row_late += detect_pattern_soft(result, '11111')
print(f"Number of times an employee works 3 late days in a row: {three_days_in_a_row_late}")
print(f"Number of times an employee works 4 late days in a row: {four_days_in_a_row_late}")
print(f"Number of times an employee works 5 late days in a row: {five_days_in_a_row_late}")
print()

In [ ]:
# 5 late shifts in two weeks maximum


In [ ]:
# Work maximum of 5 days in a row
#Max 1 midnight in a row
five_days_in_a_row = 0
six_days_in_a_row = 0
seven_days_in_a_row = 0
for key, result in days_worked_results.items():
    five_days_in_a_row += detect_pattern_soft(result, '11111')
    six_days_in_a_row += detect_pattern_soft(result, '111111')
    seven_days_in_a_row += detect_pattern_soft(result, '1111111')
print(f"Number of times an employee works 5 days in a row: {five_days_in_a_row}")
print(f"Number of times an employee works 6 days in a row: {six_days_in_a_row}")
print(f"Number of times an employee works 7 days in a row: {seven_days_in_a_row}")
print()

In [ ]:
# Avoid FT shifts (0730,1530) on consecutive days
two_ft_days_in_a_row = 0
for key, result in ft_worked_results.items():
    if ft_only[key]:
        continue
    two_ft_days_in_a_row += detect_pattern_soft(result, '11')
print(f"Number of times an employee works 2 ft shifts in a row: {two_ft_days_in_a_row}")
print()

In [ ]:
print("SOFT SUMMARY")
print()
print(f"Number of times shifts jump forwards 2.5 hours: {jumps_forward_25}")
print(f"Number of times shifts jump backwards 1.5 hours: {jumps_forward_25}")
print()
print(f"Number of times an employee gets 2 days off after midnights: {two_days_after_midnight}")
print(f"Number of times an employee gets 3 days off after midnights: {three_days_after_midnight}")
print(f"Number of times an employee gets 4 days off after midnights: {four_days_after_midnight}")
print(f"Number of times an employee gets 5 days off after midnights: {five_days_after_midnight}")
print()
print(f"Number of times an employee works 2 midnights in a row: {two_days_after_midnight}")
print()
print(f"Number of times there is a day shift within 1 days of a late shift: {one_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 2 days of a late shift: {two_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 3 days of a late shift: {three_days_off_from_late_to_day}")
print()
print(f"Number of times there is a afternoon shift within 1 days of a late shift: {one_days_off_from_late_to_afternoon}")
print(f"Number of times there is a afternoon shift within 2 days of a late shift: {two_days_off_from_late_to_afternoon}")
print()
print(f"Number of times an employee works 3 late days in a row: {three_days_in_a_row_late}")
print(f"Number of times an employee works 4 late days in a row: {four_days_in_a_row_late}")
print(f"Number of times an employee works 5 late days in a row: {five_days_in_a_row_late}")
print()
print(f"Number of times an employee works 5 days in a row: {five_days_in_a_row}")
print(f"Number of times an employee works 6 days in a row: {six_days_in_a_row}")
print(f"Number of times an employee works 7 days in a row: {seven_days_in_a_row}")
print()
print(f"Number of times an employee works 2 ft shifts in a row: {two_ft_days_in_a_row}")
print()

## Goals

In [ ]:
weekend_distance = 0
for m in staff:
    value = sum([solver.Value(days_assigned[(m, d)]) for d in weekends])
    print(f"{staff_list[m]} works {value} weekends")
    weekend_distance += abs(triangle_costs(len(shifts), len(weekends), len(staff)) - value)

print(f"Distance days from target weekends: {weekend_distance}")

In [ ]:
split_weekends = 0
for m in staff:
    sats = [solver.Value(days_assigned[m, d]) for d in saturdays]
    suns = [solver.Value(days_assigned[m, d]) for d in sundays]
    for d in sats:
        if d < len(suns) and sats[d] and suns[d]:
            split_weekends += 1

print(f"Number of split weekends: {split_weekends}")

In [ ]:
night_distance = 0
for m in staff:
    value = sum([solver.Value(midnight_shifts_assigned[(m, d)]) for d in days])
    print(f"{staff_list[m]} works {value} nights")
    night_distance += abs(triangle_costs(len(midnight_shifts), len(days), len(staff)) - value)

print(f"Distance days from target night shifts: {night_distance}")

In [ ]:
weekends_after_working_fri = 0
for m in staff:
    for d in saturdays:
        sats = solver.Value(days_assigned[m, d])
        if d and sats:
            fris = sum([solver.Value(late_shifts_assigned[m, d-1])] + [solver.Value(midnight_shifts_assigned[m, d-1])])
            if fris:
                weekends_after_working_fri += 1

print(f"Number of weekends after working past 5 on friday: {weekends_after_working_fri}")

In [ ]:
late_distance = 0
for m in staff:
    value = sum([solver.Value(late_shifts_assigned[(m, d)]) for d in days])
    print(f"{staff_list[m]} works {value} lates")
    late_distance += abs(triangle_costs(len(late_shifts), len(days), len(staff)) - value)

print(f"Distance days from target late shifts: {late_distance}")

In [ ]:
day_distance = 0
for m in staff:
    value = sum([solver.Value(day_shifts_assigned[(m, d)]) for d in days])
    print(f"{staff_list[m]} works {value} days")
    day_distance += abs(triangle_costs(len(day_shifts), len(days), len(staff)) - value)

print(f"Distance days from target day shifts: {day_distance}")

In [ ]:
afternoon_distance = 0
for m in staff:
    value = sum([solver.Value(afternoon_shifts_assigned[(m, d)]) for d in days])
    print(f"{staff_list[m]} works {value} afternoons")
    afternoon_distance += abs(triangle_costs(len(afternoon_shifts), len(days), len(staff)) - value)

print(f"Distance days from target afternoon shifts: {afternoon_distance}")

In [ ]:
weekday_distance = 0
for m in staff:
    value = sum([solver.Value(days_assigned[(m, d)]) for d in weekdays])
    print(f"{staff_list[m]} works {value} weekdays")
    weekday_distance += abs(triangle_costs(len(shifts), len(weekdays), len(staff)) - value)

print(f"Distance days from target weekdays: {weekday_distance}")

In [ ]:
print("GOAL SUMMARY")
print()
print(f"Distance days from target weekends: {weekend_distance}")
print()
print(f"Number of split weekends: {split_weekends}")
print()
print(f"Distance days from target night shifts: {night_distance}")
print()
print(f"Number of weekends after working past 5 on friday: {weekends_after_working_fri}")
print()
print(f"Distance days from target late shifts: {late_distance}")
print()
print(f"Distance days from target day shifts: {day_distance}")
print()
print(f"Distance days from target afternoon shifts: {afternoon_distance}")
print()
print(f"Distance days from target weekdays: {weekday_distance}")
print()

In [ ]:
new_works = {k:v for (k,v) in works.items() if k[1] >= 0}
save_shifts_to_file(works, len(staff_list), jan[1], len(shift_list), solver)